### Tutor to generate question from a given book's section

In [10]:
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="asia-southeast1"

FOLDER_NAME="."

In [7]:
%pip install aifs

  Using cached aifs-0.0.16-py3-none-any.whl.metadata (3.1 kB)
Using cached aifs-0.0.16-py3-none-any.whl (54 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
from aifs import search
from langchain.tools import tool

In [9]:
@tool
def File_search(query: str) -> str:
    """
    Searches document folder for relevant document excerpts.
    
    Args:
        query for docs folder.
    
    Returns:
        str: Document execerpts.
    """
    name: str = "story_file_search"
    description: str = (
        "A tool to search for ACME information"
        "Useful for when you need to answer questions about ACME programs. "
        "Input should be a search query."
    )
    result = search(
        query=query,
        #replace path to your file below with your file path
        path="./chapter1.pdf",
        max_results=5
    )
    print(result)
    return result


In [11]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool
# from langchain_vertexai import ChatGemini
from crewai_tools.tools import FileReadTool
import os, requests, re, mdpdf, subprocess
from vertexai.preview.vision_models import ImageGenerationModel
from langchain_google_vertexai import ChatVertexAI
import uuid, os

# Initialize Gemini LLM
llm = ChatVertexAI(
    model_name="gemini-pro-1.5", # Replace with your desired Gemini model
    project_id=os.getenv(PROJECT_ID), # Your Vertex AI project ID
    location="us-central1", # Your Vertex AI location
)


In [12]:

import os
from crewai import Agent, Task, Crew, Process

#You can choose to use a local model through Ollama for example.
#
# from langchain_community.llms import Ollama
# ollama_llm = Ollama(model="openhermes")

#Install duckduckgo-search for this example:
#!pip install -U duckduckgo-search

from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

In [14]:


# Define your agents with roles and goals
researcher = Agent(
  role='researcher',
  goal='Summarize info from file_search',
  backstory="""You are good at using file_search to search docs and summarize info.""",
  verbose=True,
  allow_delegation=True,
  tools=[File_search],
  llm=llm
  
  
  # You can pass an optional llm attribute specifying what mode you wanna use.
  # It can be a local model through Ollama / LM Studio or a remote
  # model like vertexai, Mistral, Antrophic of others (https://python.langchain.com/docs/integrations/llms/)
  #  # Examples:xx
  
)

writer = Agent(
  role='ACME Programs Background Info Writer',
  goal='Craft reports with information about ACME programs',
  backstory="""You are a renowned wrier with years experience, known for
  your insightful skills and extensive experience working in the domain.""",
  verbose=True,
  allow_delegation=True,
  llm=llm,
  max_iter=200,
  max_rpm=20
)



task1 = Task(
  description="""Use the document search tool to Conduct a comprehensive analysis of the ACME information in the document library. You must use the search tool provided, information should be from documents searched with that tool.
  Your final answer MUST be a full list of information related to ACME, the organizations associated, and the schedules associated. Include some verbatim excerpts from any document searches. Use the File_search tool.""",
  agent=researcher,
  tools=[File_search]
)

task2 = Task(
  description="""Using the insights provided, develop an engaging report that explains information related to ACME.
  Your post should be informative yet accessible, catering to the audience.
  Make it sound professional.
  Your final answer MUST be a complete reporting of the information with at least 4 paragraphs.""",
  agent=writer
  

)

# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  #agents=[researcher1],
  tasks=[task1, task2],
  #tasks=[task0],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)
# @analyticsrepo01
# Comment

ValidationError: 1 validation error for Task
expected_output
  Field required [type=missing, input_value={'description': 'Use the ...ch at 0x7fa5b3d6cae0>)]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing